In [1]:
!pip install Faker

In [2]:
NUM_OF_BILLS = 12
NUM_OF_ROWS = 1000
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'Novemeber', 'December']

In [3]:
import pandas as pd
import numpy as np
from faker import Faker
import datetime
import random
# import matplotlib.pyplot as plt
# import seaborn as sns

In [4]:
from faker import Faker
import datetime
import random

## Demographics

In [5]:
f = Faker()

In [6]:
names = [f.name() for _ in range(NUM_OF_ROWS)]

In [7]:
demographics = pd.DataFrame(data=names).rename(columns={0: 'name'})

### Birth Dates

In [8]:
def gen_date():
    start_date = datetime.date(1960, 1, 1)
    end_date = datetime.date(2000, 1, 1)
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    random_date = start_date + datetime.timedelta(days=random_number_of_days)
    return random_date

In [9]:
dates = [gen_date() for _ in range(NUM_OF_ROWS)]

In [10]:
demographics['birth_date'] = dates

In [11]:
dist_27 = lambda x: np.random.choice([0,1], p=[0.85, 0.15])
dist_35 = lambda x: np.random.choice([0,1,2,3], p=[0.1, 0.4, 0.3, .2])
dist_70 = lambda x: np.random.choice([0,1,2,3,4,5], p=[0.03, 0.3, 0.4, .2, .05, .02])

In [12]:
demographics['age'] = demographics.birth_date.apply(lambda x: datetime.datetime.now().year - x.year)

### Children

In [13]:
func = lambda x: dist_27(_) if x<=27 else(dist_35(_) if x<=35 else dist_70(_))

demographics['children'] = demographics.age.apply(func)

### Employment

In [14]:
working = lambda x: np.random.choice([0,1], p=[0.1, 0.9])
working_lst = [working(_) for _ in range(NUM_OF_ROWS)]

In [15]:
demographics['employment'] = working_lst

### ID

In [16]:
id = np.random.choice(np.arange(210000,470000), size=NUM_OF_ROWS)

In [17]:
demographics['id'] = id

### Cities

In [18]:
cities = ['Tel_Aviv_Yaffo', 'Rishon_LeTsiyon', 'Ramat_Gan', 'Petah_Tikva', 'Ramat_HaSharon', 'Petah_Tikva', 'Herzliya']
weights = [.5, .15, .1, .1, .05, .075, .025]
assert len(weights) == len(cities)
assert sum(weights) == 1

In [19]:
cities_lst = np.random.choice(cities, p=weights, size=NUM_OF_ROWS)

In [20]:
demographics['city'] = cities_lst

### Email and Password

In [21]:
emails = [f.ascii_email() for _ in range(NUM_OF_ROWS)]
demographics['email'] = emails

In [22]:
passwords = [f.password() for _ in range(NUM_OF_ROWS)]
demographics['password'] = passwords

In [23]:
demographics

,name,birth_date,age,children,employment,id,city,email,password
0,Mr. Robert Davenport DDS,1963-11-26,59,2,1,232038,Tel_Aviv_Yaffo,jefferytownsend@hutchinson.info,)&5IOAvscQ
1,Amber Elliott,1971-05-30,51,2,1,346387,Tel_Aviv_Yaffo,walkergina@richmond.com,@J4W2TAlx(
2,Rodney Pacheco,1999-12-24,23,0,1,329159,Tel_Aviv_Yaffo,pmathis@mills.com,!+h0YxSiT9
3,Jonathan Gonzalez,1993-07-17,29,3,1,328165,Petah_Tikva,frhodes@gmail.com,4v7evJgxA@
4,Jose Pineda,1974-07-18,48,5,1,242816,Tel_Aviv_Yaffo,woodnicole@abbott.com,P4bXDE8t%A
...,...,...,...,...,...,...,...,...,...
995,Jessica Thompson,1994-12-15,28,2,0,422493,Tel_Aviv_Yaffo,zwilliams@yahoo.com,9MlBaqNZ*A
996,Audrey Brown,1970-07-19,52,3,0,282307,Rishon_LeTsiyon,qmurray@yahoo.com,tWiQ9E6to_
997,Steven Bennett,1984-06-19,38,2,1,252693,Rishon_LeTsiyon,ashah@hotmail.com,(n)3VuLdAa
998,Patrick Edwards,1979-08-23,43,1,1,349423,Petah_Tikva,bnelson@yahoo.com,V7IWl3Ga&y


### Seeding the clusters

In [24]:
demographics.loc[:, 'money'] = 'rich'
demographics.loc[:666, 'money'] = 'medium'
demographics.loc[:300, 'money'] = 'poor'

In [25]:
# sns.countplot(data=demographics, x='city', hue='money')

In [26]:
map_ = {'poor': [0,1], 'medium': [1,2,3,4], 'rich': [4,5]}

demographics['seed'] = demographics.money.apply(lambda x: np.random.choice(map_[x]))

## Bills

In [27]:
def get_mean_scale(means, stds, i, children_value=40):
    mean = means[demographics.loc[i, 'seed']] + demographics.loc[i, 'seed']*demographics.loc[i, 'children']*children_value
    std = stds[demographics.loc[i, 'seed']]
    return mean, std

### Electricity

In [28]:
# for electricity 

means_elec = [120, 160, 210, 260, 320, 460]
stds = [20, 25, 30, 35, 50, 70]

mean_and_stds = [get_mean_scale(means_elec, stds, i, 40) for i in range(NUM_OF_ROWS)]
lst = [np.random.normal(loc=mean, scale=std, size=NUM_OF_BILLS) for (mean,std) in mean_and_stds]

In [29]:
# sort bills so most expensive in winter and then summer

map_ = 11 - np.array([0, 1, 11, 7, 6, 5, 3, 10, 4, 9, 8, 2])
A = np.sort(lst, axis=1)
B = np.array([map_ for _ in range(NUM_OF_ROWS)])

bills = np.take_along_axis(A,B,1)

In [30]:
electricity = demographics[['name', 'id']]

for i in range(NUM_OF_BILLS):
    electricity.insert(electricity.shape[1], months[i%12], bills[:, i])

### Water bills

In [31]:
means_water = np.array(means_elec)/1.5 + 10
# means

mean_and_stds = [get_mean_scale(means_water, stds, i, 10) for i in range(len(demographics))]
lst = [np.random.normal(loc=mean, scale=std, size=NUM_OF_BILLS) for (mean,std) in mean_and_stds]

# lst = [np.random.normal(loc=means[demographics.loc[i, 'seed']] + demographics.loc[i, 'seed']*demographics.loc[i, 'children']*10, scale=stds[demographics.loc[i, 'seed']], size=12) for i in range(len(df))]
bills = np.abs(np.array(lst))

In [32]:
water = demographics[['name', 'id']]

for i in range(NUM_OF_BILLS):
#     df_water[months[i]] = bills[:, i]
    water.insert(water.shape[1], months[i%12], bills[:, i])

### Arnona

In [33]:
cities

['Tel_Aviv_Yaffo',
 'Rishon_LeTsiyon',
 'Ramat_Gan',
 'Petah_Tikva',
 'Ramat_HaSharon',
 'Petah_Tikva',
 'Herzliya']

In [34]:
arnona_prices = [1000, 850, 650, 550, 600, 400, 400]
# lst = [np.random.normal(loc=means[df.loc[i, 'seed']] + df.loc[i, 'seed']*df.loc[i, 'children']*10, scale=stds[df.loc[i, 'seed']], size=12) for i in range(len(df))]

In [35]:
# i+=1

# mean_and_stds = [get_mean_scale(arnona, stds, i, 10) for i in range(NUM_OF_BILLS)]
# arnona_lst = [np.random.normal(loc=mean, scale=100, size=NUM_OF_BILLS) for (mean,_) in mean_and_stds]

# arnona_lst = [np.random.normal(loc=arnona[cities.index(demographics.loc[i, 'city'])]+demographics.loc[i, 'seed']*demographics.loc[i, 'children']*10, scale=100) for i in range(len(demographics))]

arnona_lst = []

for i in range(len(demographics)):
    arnona = arnona_prices[cities.index(demographics.loc[i, 'city'])]
    arnona += demographics.loc[i, 'seed']*demographics.loc[i, 'children']*10
    res = np.random.normal(loc=arnona, scale=100)
    arnona_lst.append(res)

In [36]:
arnona = demographics[['name', 'id']]
arnona.insert(arnona.shape[1], 'price', arnona_lst)

## Signed Up

In [37]:
signed_up = [1 for _ in range(950)] + [0 for _ in range(50)]

random.shuffle(signed_up)

In [38]:
demographics.insert(demographics.shape[1], 'Signed_Up', signed_up)
electricity.insert(electricity.shape[1], 'Signed_Up', signed_up)
water.insert(water.shape[1], 'Signed_Up', signed_up)
arnona.insert(arnona.shape[1], 'Signed_Up', signed_up)

## Save it all

In [39]:
demographics.drop(['money', 'seed'], axis=1).to_csv('demographics.csv')
electricity.to_csv('electricity.csv')
water.to_csv('water.csv')
arnona.to_csv('arnona.csv')